# IBM Applied Data Science Capstone Course by Coursera

## Week 3

### Part 1: Collecting Toronto data and building a pandas dataframe

#### 1. Imports

In [1]:
import pandas as pd
import numpy as np

import json

from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests

from sklearn.cluster import KMeans

#### 2. Read wiki data into pandas dataframe

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 3. Process the dataframe

###### 3.1 Rename columns

In [3]:
df.rename(columns={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


###### 3.2 Drop rows that don't have assigned borough

In [4]:
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


###### 3.3 Group rows with same postal code and concatenate neighborhood names 

In [5]:
grouped_df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
grouped_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### 3.4 Process neighborhoods that are not assigned 
Set borough's name as name of the neighborhood

In [6]:
grouped_df['Neighborhood'].replace('Not assigned', df['Borough'], inplace=True)
grouped_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
grouped_df.shape

(103, 3)

### Part 2: Getting coordinates and extending the dataframe

#### 1. Downloading geospatial data

In [8]:
coors = pd.read_csv('https://cocl.us/Geospatial_data')
print(coors.shape)
print(coors.head())

(103, 3)
  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


#### 2. Merge the two dataframes

##### 2.1 Set indexes on both dataframes on the PostalCode column in order to be sure frames will be correctly merged

In [9]:
toronto_df_indexed = grouped_df.set_index('PostalCode')
coors_indexed = coors.set_index('Postal Code')
toronto_coors_df = pd.concat([toronto_df_indexed, coors_indexed], axis=1, join='inner')
toronto_coors_df.head()

,Borough,Neighborhood,Latitude,Longitude
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


##### 2.2 Reset index column's name

In [10]:
toronto_coors_df.index.name = 'PostalCode'
toronto_coors_df.reset_index(inplace=True)

##### 2.3 Check the new dataframe with coordinates

In [11]:
print(toronto_coors_df.shape)
toronto_coors_df.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Part 3: Explore and cluster neighbourhoods of Toronto

In [12]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [13]:
toronto_borough_df = toronto_coors_df[toronto_coors_df['Borough'].str.contains('Toronto')]
print(toronto_borough_df.shape)
toronto_borough_df.head()

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [14]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, post, borough, neigh in zip(toronto_borough_df['Latitude'], toronto_borough_df['Longitude'], toronto_borough_df['PostalCode'], toronto_borough_df['Borough'], toronto_borough_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
    
toronto_map

In [15]:
CLIENT_ID = 'XXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXX' # your Foursquare Secret
VERSION = '20190906' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print('Venues loadded')
    return(nearby_venues)

In [17]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=toronto_borough_df['Neighborhood'],
                                latitudes=toronto_borough_df['Latitude'],
                                longitudes=toronto_borough_df['Longitude'])

Venues loadded


In [18]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


As venues contain neighbhorhood category we rename dataframes column to avoid collisions

In [19]:
toronto_venues.rename(columns={'Neighborhood':'Neighbourhood'}, inplace=True)

Get venues category list

In [20]:
categories_list = toronto_venues['Venue Category'].unique()

One hot encoding

In [22]:
base_features = toronto_venues[['Neighbourhood', 'Neighborhood Latitude', 'Neighborhood Longitude']]
df_dummies = pd.get_dummies(toronto_venues['Venue Category'])

# merge the 2 dataframes into a new one
toronto_onehot_df = pd.concat([base_features, df_dummies], axis=1)
toronto_onehot_df.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,43.676357,-79.293031,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,43.676357,-79.293031,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,43.676357,-79.293031,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,43.676357,-79.293031,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",43.679557,-79.352188,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group data by neighbourhood and get the number of venues of each category for each neighborhood

In [78]:
toronto_grouped_df = toronto_onehot_df.groupby(['Neighbourhood', 'Neighborhood Latitude', 'Neighborhood Longitude'], as_index=True)[categories_list].sum().reset_index()
toronto_grouped_df

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Trail,Health Food Store,Pub,Neighborhood,Greek Restaurant,Ice Cream Shop,Cosmetics Shop,...,Flea Market,Cajun / Creole Restaurant,Bus Stop,Food,Indie Movie Theater,Post Office,Skate Park,Garden Center,Auto Workshop,Recording Studio
0,"Adelaide, King, Richmond",43.650571,-79.384568,0,0,0,1,1,1,3,...,0,0,0,0,0,0,0,0,0,0
1,Berczy Park,43.644771,-79.373306,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Cabbagetown, St. James Town",43.667967,-79.367675,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Central Bay Street,43.657952,-79.387383,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
7,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,Christie,43.669542,-79.422564,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Church and Wellesley,43.665860,-79.383160,0,0,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
# Borrow the function from the lab. Sort the venues in descending order
def get_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)

# drop latitude and longitude and copy neighbourhood data to the sorted dataframe
toronto_venues_categories = toronto_grouped_df.drop(['Neighborhood Latitude', 'Neighborhood Longitude'], 1)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_venues_categories['Neighbourhood']

# sort each neighbourhood's venue categories by most occurencies
for row in np.arange(toronto_venues_categories.shape[0]):
    neighborhoods_venues_sorted.iloc[row, 1:] = get_most_common_venues(toronto_venues_categories.iloc[row, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Burger Joint,Cosmetics Shop,American Restaurant,Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Cheese Shop,Steakhouse,Seafood Restaurant,Beer Bar,Italian Restaurant,Bakery,Café
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Climbing Gym,Bakery,Grocery Store,Furniture / Home Store,Bar,Convenience Store,Music Venue
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Recording Studio,Restaurant,Butcher,Fast Food Restaurant,Garden,Spa,Smoke Shop,Burrito Place,Farmers Market
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Gate,Boat or Ferry,Bar,Sculpture Garden,Airport Terminal,Coffee Shop,Airport,Boutique
5,"Cabbagetown, St. James Town",Coffee Shop,Italian Restaurant,Market,Café,Pizza Place,Bakery,Restaurant,Park,Pub,Deli / Bodega
6,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Middle Eastern Restaurant,Ice Cream Shop,Dessert Shop,Spa,Bar
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Dumpling Restaurant,Mexican Restaurant,Bakery,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,Comfort Food Restaurant
8,Christie,Café,Grocery Store,Park,Nightclub,Coffee Shop,Diner,Baby Store,Restaurant,Athletics & Sports,Convenience Store
9,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Gastropub,Café,Mediterranean Restaurant,Men's Store,Bubble Tea Shop


Cluster data by venue categories

In [75]:
k=3
clusters = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_df.drop(['Neighbourhood', 'Neighborhood Latitude', 'Neighborhood Longitude'], 1))

Add clusters to dataframe

In [76]:
toronto_grouped_df['Cluster'] = clusters.labels_
toronto_grouped_df.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Trail,Health Food Store,Pub,Neighborhood,Greek Restaurant,Ice Cream Shop,Cosmetics Shop,...,Cajun / Creole Restaurant,Bus Stop,Food,Indie Movie Theater,Post Office,Skate Park,Garden Center,Auto Workshop,Recording Studio,Cluster
0,"Adelaide, King, Richmond",43.650571,-79.384568,0,0,0,1,1,1,3,...,0,0,0,0,0,0,0,0,0,1
1,Berczy Park,43.644771,-79.373306,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, bor, cluster in zip(toronto_grouped_df['Neighborhood Latitude'], toronto_grouped_df['Neighborhood Longitude'], toronto_grouped_df['Neighbourhood'], toronto_grouped_df['Cluster']):
    label = folium.Popup('{} : Cluster {}'.format(bor, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters